In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 

In [ ]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv(PATH+"//Data one hot encoded.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)
df1["EntryDate"] = pd.to_datetime(df1['EntryDate'])
df1["ExitDate"] = pd.to_datetime(df1['ExitDate'])
df1 = df1.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)


In [ ]:
# for each step and project type
# preparing the effective path for each client with aggregated features f 
data = pd.DataFrame()
for i in range(len(clientID)):
    temp = df[df["ClientID"]==clientID[i]].reset_index(drop=True)
    temp1 = df1[df1["ClientID"]==clientID[i]].reset_index(drop=True)
    p_unique = temp['ProjectType'].unique()
    
    #for each efective path step, considering the features fo the first appearance of the project type only
    for k in range(len(p_unique)):
        #taking out the features
        temp3 = temp1[temp1['ProjectType']==p_unique[k]].drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 'ProjectType',
       'ExitID', 'ExitDate', 'HouseholdID'],axis=1).iloc[0]
        #taking out the project type
        ptype = pd.DataFrame({'Step':k+1,'ClientID':clientID[i],"ProjectType":p_unique[k]},index=[0]).T
        #combining the project type and the averaged feature
        temp4 = pd.concat([ptype,temp3],axis = 0).T
        data = data.append(temp4).reset_index(drop=True)
data.to_csv(PATH+"//Data for cosine similarity with effective path and mean aggregated feature-project type.csv")
data = pd.read_csv(PATH+"//Data for cosine similarity with effective path and mean aggregated feature-project type.csv").drop(['Unnamed: 0'],axis=1)



In [ ]:
# separating the living situation and the exit destination into categories
goal = [10,11]
closer = [3,19,20,21,22,23,25,26,28,31]
trans = [1,2,4,5,6,12,13,14,15,18,27,29]
no_progress = [7,16]
hard_to_judge = [8,9,17,24,30,99]

In [ ]:
# for each target
# preparing the feature set by cosindering the features of the last step as the feature of the target
data_target = pd.DataFrame()
for i in range(len(clientID)):
    temp = df[df["ClientID"]==clientID[i]].reset_index(drop=True)
    temp1 = df1[df1["ClientID"]==clientID[i]].reset_index(drop=True)
    if temp['Destination'].iloc[-1] in goal:
        temp3 = temp1.drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 'ProjectType','ExitID', 'ExitDate', 'HouseholdID'],axis=1).iloc[-1]
        ttype = pd.DataFrame({'ClientID':clientID[i],"target":'ultimate goal'},index=[0]).T
        temp4 = pd.concat([ttype,temp3],axis = 0).T
        data_target = data_target.append(temp4).reset_index(drop=True)
    elif temp['Destination'].iloc[-1] in  closer:
        temp3 = temp1.drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 'ProjectType','ExitID', 'ExitDate', 'HouseholdID'],axis=1).iloc[-1]
        ttype = pd.DataFrame({'ClientID':clientID[i],"target":'closer to exit'},index=[0]).T
        temp4 = pd.concat([ttype,temp3],axis = 0).T
        data_target = data_target.append(temp4).reset_index(drop=True)
    elif temp['Destination'].iloc[-1] in trans:
        temp3 = temp1.drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 'ProjectType','ExitID', 'ExitDate', 'HouseholdID'],axis=1).iloc[-1]
        ttype = pd.DataFrame({'ClientID':clientID[i],"target":'transitional phase'},index=[0]).T
        temp4 = pd.concat([ttype,temp3],axis = 0).T
        data_target = data_target.append(temp4).reset_index(drop=True)
    elif temp['Destination'].iloc[-1] in no_progress:
        temp3 = temp1.drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 'ProjectType','ExitID', 'ExitDate', 'HouseholdID'],axis=1).iloc[-1]
        ttype = pd.DataFrame({'ClientID':clientID[i],"target":'no progress or worse'},index=[0]).T
        temp4 = pd.concat([ttype,temp3],axis = 0).T
        data_target = data_target.append(temp4).reset_index(drop=True)
    elif temp['Destination'].iloc[-1] in hard_to_judge:
        temp3 = temp1.drop(['EnrollmentID', 'ClientID', 'ProjectID', 'EntryDate', 'ProjectType','ExitID', 'ExitDate', 'HouseholdID'],axis=1).iloc[-1]
        ttype = pd.DataFrame({'ClientID':clientID[i],"target":'hard to judge'},index=[0]).T
        temp4 = pd.concat([ttype,temp3],axis = 0).T
        data_target = data_target.append(temp4).reset_index(drop=True)
    
data_target.to_csv(PATH+"//Data for cosine similarity with effective path and mean aggregated feature-target.csv")
data_target = pd.read_csv(PATH+"//Data for cosine similarity with effective path and mean aggregated feature-target.csv").drop(['Unnamed: 0'],axis=1)




In [ ]:
project = data.groupby(['Step','ProjectType']).sum()    
target = data_target.groupby('target').sum()

In [ ]:
F = pd.DataFrame()
for i in data.Step.unique():
    temp = project.loc[i]
    temp = temp.append(target).reset_index()
    temp['Step'] = [i for k in range(len(temp))]
    #combining the step and the TF-IDF
    F = F.append(temp)
F = F.groupby(['Step',"index"]).sum()

    

In [ ]:
# calculating tf_idf for each step 
t = ["ultimate goal","closer to exit","transitional phase","no progress or worse","hard to judge"]
tf_idf_norm_combo = pd.DataFrame()

a = 1
for a in data.Step.unique():
    print(a)
    F1 = F.loc[a]
    p = [x for x in F1.index]
    P = len(p)
    tf_idf = pd.DataFrame([],index = F1.index, columns = F1.columns)
    for i in range(len(F1.index)):
        for j in range(len(F1.columns)):
            temp = F1[F1.columns[j]]
            tf = temp.loc[F1.index[i]]
            n = len(temp[temp!=0])+1
            tf_idf.loc[F1.index[i]][F1.columns[j]] = tf * max(0,np.log(P/n))

    tf_idf_norm = pd.DataFrame([],index = tf_idf.index, columns = tf_idf.columns)
    for i in  p:
        if np.sqrt(np.square(tf_idf).sum(axis=1)).loc[i] == 0:
                tf_idf_norm.loc[i] = 0
        else:
            tf_idf_norm.loc[i] = tf_idf.loc[i]/np.sqrt(np.square(tf_idf).sum(axis=1)).loc[i]

    tf_idf_norm['Step'] = [a for k in range(len(tf_idf_norm))]
    #combining the step and the TF-IDF
    tf_idf_norm_combo = tf_idf_norm_combo.append(tf_idf_norm)
tf_idf_norm_combo = tf_idf_norm_combo.groupby(['Step',"index"]).sum()
tf_idf_norm_combo.to_csv(PATH+"//Normalized TF_IDF for project type and target for each step of effective path.csv")


